In [30]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

In [4]:
from datasets import load_dataset
import cassio   
from PyPDF2 import PdfReader

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()
ASTRA_DB_APPLICATION_TOKEN=os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_ID=os.getenv("ASTRA_DB_ID")
GROQ_API=os.getenv("GROQ_API")

In [17]:
pdfreader=PdfReader('Artificial_intelligence.pdf')

In [18]:
from typing_extensions import Concatenate
raw_text=' '
for i,page in enumerate(pdfreader.pages):
    content=page.extract_text()
    if content:
        raw_text+=content

In [26]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [31]:
llm=ChatGroq(model='gemma2-9b-it',)
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en", model_kwargs={"device": "cpu"}, encode_kwargs={"normalize_embeddings": True}
)

In [33]:
astra_vector_store=Cassandra(embedding=embeddings,table_name="qa_mini_demo",session=None,keyspace=None)

In [41]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter=CharacterTextSplitter(separator='\n',chunk_size=800,chunk_overlap=200,length_function=len)
texts=text_splitter.split_text(raw_text)

In [57]:
texts[:50]

['Artificial intelligence\nArtificial intelligence  (AI) is the capability of computational systems  to perform tasks typically\nassociated with human intelligence , such as learning, reasoning, proble m-solving, perception, and\ndecision-making. It is a field of research  in computer science  that develops and studies methods and\nsoftware  that enable machines to perceive their environment  and use learning  and intelligence  to take\nactions that maximize their chances of achieving defined goals.[1] There is debate  over whether artificial\nintelligence exhibits genuine intelligence  or merely simulates it by imitating human-like behaviors.[2]\nHigh-profile applications of AI include advanced web search engines  (e.g., Google Search );',
 'intelligence exhibits genuine intelligence  or merely simulates it by imitating human-like behaviors.[2]\nHigh-profile applications of AI include advanced web search engines  (e.g., Google Search );\nrecommendation systems  (used by YouTube, Amazo

In [56]:
astra_vector_store.add_texts(texts[:50])
astra_vector_index=VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [60]:
F=True
while True: 
    if F:
        query_text=input("Enter text").strip()
    else:
        query_text=input("Enter next Question").strip()
    if query_text.lower()=='exit':
        break 
    if query_text == " ":
        continue
    F=False
    print('Question:',query_text)
    answer=astra_vector_index.query(query_text,llm=llm).strip()
    print("ANSWER: ",answer)
    for doc,score in astra_vector_store.similarity_search_with_relevance_scores(query=query_text,k=2):
        print(f"{score:.2f},{doc.page_content[:80]}")

Question: What is AI
ANSWER:  Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making.
0.93,Artificial intelligence
Artificial intelligence  (AI) is the capability of compu
0.93,Artificial intelligence
Artificial intelligence  (AI) is the capability of compu
Question: Wat are core parts of AI
ANSWER:  According to the text, what are some tools AI researchers have devised to solve problems related to incomplete or uncertain information?


The text states that AI researchers have devised tools using methods from probability theory and economics.  It also mentions  decision theory, decision analysis, and information value theory as precise mathematical tools used for analyzing choices and planning.
0.92,vague and partially true.[87]
Non-monotonic logics , including logic programming
0.92,vague and partially true.[87]
Non-monotonic log